Here we will be working with the U.S. Education Dataset from Kaggle. The data gives detailed state level information on several facets of the state of education on an annual basis. To learn more about the data and the column descriptions, click the Kaggle link above.

Access this data from the Thinkful database using the following credentials:

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [ ]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

df0 = pd.read_sql_query('select * from useducation',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [ ]:
# Import the data into two different pandas for later parallel treatments
df = df0.copy()
df1 = df0.copy()

# 1. Determine all the variable types and find the fraction of the missing values for each variable.

In [ ]:
# Help functions to gather basic descriptions
def describe(df):
    return pd.concat([df.describe().T,
                      df.mad().rename('mean abs dev'),
                      df.skew().rename('skew'),
                      df.kurt().rename('kurt'),
                      df.nunique().rename('unique')
                     ], axis=1).T

In [ ]:
describe(df)

## Numerical data types

In [ ]:
# Numerical describe
df_stat = df.describe(include = [np.number]) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

### Many of columns have missing value that we will address later

## Object data type

In [ ]:
# Non-numerical describe
df_stat = df.describe(include = ['O']) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

### Two of the columns have string data types

# 2. Choose a strategy to deal with the missing values for each variables. For which variables would filling in the missing values with some value make sense? For which might tossing out the records entirely make sense?

#### Assuming no timeseries effect here, then following strategies are selected:
#### Interpolate: 'STATE',
#### Select mean: 'ENROLL', 'FEDERAL_REVENUE', 'STATE_REVENUE', 'LOCAL_REVENUE','INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE','CAPITAL_OUTLAY_EXPENDITURE', 'GRADES_PK_G', 'GRADES_KG_G','GRADES_4_G', 'GRADES_8_G', 'GRADES_12_G', 'GRADES_1_8_G','GRADES_9_12_G', 'GRADES_ALL_G', 
#### Interpolate: 'AVG_MATH_4_SCORE', 'AVG_MATH_8_SCORE','AVG_READING_4_SCORE', 'AVG_READING_8_SCORE'],
#### Drop: 'TOTAL_REVENUE', 'TOTAL_EXPENDITURE'
#### Custom: After having above created then we can create new Primary Key

# 3. Now, take into account the time factor. Replicate your second answer but this time fill in the missing values by using a statistic that is calculated within the year of the observation. For example, if you want to fill a missing value for a variable with the mean of that variable, calculate the mean by using only the observations for that specific year.

## Fill NAs with mean
### First we use mean within each year. And if there is no values within a year we use State as a grouping criteria


In [ ]:
df.head(2)

### We use groupy and transform functions to apply mean value for all missing values

In [ ]:
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'float64' else x.fillna('NaN'))

In [ ]:
# Numerical describe
df_stat = df.describe(include = [np.number]) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

# 4. This time, fill in the missing values using interpolation (extrapolation).

In [ ]:
# Numerical describe
df_stat = df1.describe(include = [np.number]) 
df_stat.loc['dtype'] = df.dtypes
df_stat.loc['size'] = len(df)
df_stat.loc['% Null'] = df.isnull().mean()*100
df_stat

In [ ]:
# Interpolate the missing values using Linear method. Mind this method ignores the index and treat the values as equally spaced.
df = df.apply(lambda x: x.fillna(x.mean()) if x.dtype.kind in 'float64' else x.interpolate(method ='linear', limit_direction ='forward') )

# 5. Compare your results for the 2nd, 3rd, and 4th questions. Do you find any meaningful differences?

## Some of the timeseries data have cyclical characteristics, making them more suitable for mean method to fill the missing values. Linear interpolations of the missing values for these data ignore such cyclical trend. 
## Other set of data such as grades are not cyclical and using mean method for missing values are more appropriate than interpolation. Interpolation for grades may risk of biasing because of proximity to nearby grades.